# Slivka Client Demo

## Introduction

This notebook demonstrates how to use the Slivka Python client to interact with a Slivka server. You'll learn how to install the client, query available services, submit a job, and retrieve the results.

### Install Slivka Client

In [ ]:
%pip install slivka-client==1.2.1b1


### Configure Slivka Server URL

In [ ]:
# Set the Slivka server URL
local_slivka_url = 'http://slivka-bio:8000/'  # For Slivka running in Docker
dundee_slivka_url = 'https://www.compbio.dundee.ac.uk/slivka/'

# Set to True to use the local Slivka server
use_local_slivka = True

slivka_server_url = local_slivka_url if use_local_slivka else dundee_slivka_url
print(f"Using Slivka server at: {slivka_server_url}")


### Initialize Slivka Client

In [ ]:
from slivka_client import SlivkaClient

client = SlivkaClient(slivka_server_url)
print(f"Connected to Slivka server at: {client.url}")


### Query Available Services

In [ ]:
# List available services
for service in client.services:
    print(f"Service name: {service.name}, ID: {service.id}")


### Select a Service (ClustalO)


In [ ]:
service = client['clustalo']
print(f"Selected service: {service.name} (ID: {service.id})")


### Submit a Job to Align Sequences with ClustalO


In [ ]:
# Define parameters for the job
data = {
    'iterations': 1,
    'max-guidetree-iterations': 1,
    'max-hmm-iterations': 1,
}

# Specify the input file path
input_file_path = 'data/fam83b-pandda-chains-for-align-test.fa'

# Submit the job with the input file
with open(input_file_path, 'rb') as file_object:
    files = {'input': file_object}
    job = service.submit_job(data=data, files=files)
    print(f"Job submitted with ID: {job.id}")


### Monitor Job Status


In [ ]:
import time

# Poll the job status until it is completed
while job.status not in ('COMPLETED', 'FAILED'):
    print(f"Job status: {job.status}")
    time.sleep(3)

print(f"Job completed with status: {job.status}")


### Retrieve Job Results


In [ ]:
import os

# Download each file in the job results
for file in job.files:
    download_path = f"./downloads/{file.id}"
    os.makedirs(os.path.dirname(download_path), exist_ok=True)
    file.dump(download_path)
    print(f"File {file.id} downloaded to {download_path}")